# Algorithms

These are the algorithms verbatim from the paper.

In [318]:
# Algorithm 1

def combine(U_n, n, U_m, m):
    U_nm = U_n * m + U_m
    nm = n*m
    return U_nm, nm

In [319]:
# Algorithm 2

def divide(U_nm, nm, n):
    U_m = U_nm // n
    U_n = U_nm % n
    m = nm // n
    return U_m, m, U_n

In [320]:
# Algorithm 3

def sample(U_n, n, m):
    if U_n < m:
        B = 1
        x = m
        U_x = U_n
    else:
        B = 0
        x = n-m
        U_x = U_n - m
    return U_x, x, B

In [321]:
# Algorithm 4

def expand(U_x, B, m):
    if B:
        U_n = U_x
    else:
        U_n = m + U_x
    return U_n

In [322]:
import random

def fetch():
    return random.randint(0,1)

In [323]:
# Algorithm 5

def generate_uniform(U_s, s, n, N):
    while True:
        while s < N:
            U_s, s = combine(U_s, s, fetch(), 2)
        U_s, s, b = sample(U_s, s,s%n)
        if not b:
            U_s, s, U_n = divide(U_s, s, n)
            return U_s, s, U_n

In [324]:
# Algorithm 6

def generate_bernoulli(U_s, s, m, n, N):
    U_s, s, U_n = generate_uniform(U_s, s, n, N)
    U_x, x, B = sample(U_n, n, m)
    U_s, s = combine(U_s, s, U_x, x)
    return U_s, s, B

In [325]:
# Algorithm 7

def make_distribution(weights):
    outputs = []
    offsets = []
    for i, w_i in enumerate(weights):
        offsets = offsets + [len(outputs)]
        outputs = outputs + [i]*w_i
    return outputs, offsets

TODO: This does not match the paper

TODO: document the output $i$

In [326]:
# Algorithm 8

def generate_distribution(U_s, s, N, weights, outputs, offsets):
    U_s, s, U_n = generate_uniform(U_s, s, len(outputs), N)
    i = outputs[U_n]
    U_s, s = combine(U_s, s, U_n - offsets[i], weights[i])
    return U_s, s, i

In [327]:
# Algorithm 9

def combine_bernoulli(U_s, s, N, m, n, B):
    if B:
        x = m
    else:
        x = n-m    
    U_s, s, U_x = generate_uniform(U_s, s, x, N)
    U_n = expand(U_x, B, m)
    U_s, s = combine(U_s, s, U_n, n)
    return U_s, s

In [328]:
# Algorithm 10

def combine_distribution(U_s, s, N, weights, outputs, offsets, i):
    U_s, s, U_x = generate_uniform (U_s, s, weights[i], N)
    U_s, s = combine(U_s, s, U_x + offsets[i], len(outputs))
    return U_s, s

# Graphs

Here are the functions used to calculate the graphs:

In [329]:
import math

def von_neumann_uniform_loss(n):
    r = 1 # The smallest power of 2 >= than range 
    b = 0 # The number of bits in r
    while r < n:
        r *= 2
        b += 1
    return r*b/n - math.log2(n)

def knuth_yao_uniform_loss(n):
    def calculate_fdr_exact(u, depth):
        count = 0
        while u<n:
            u *= 2
            count += 1
        if n==u or depth==1:
            return count
        p = n/u
        return count + (1-p) * calculate_fdr_exact(u-n, depth-1)

    return calculate_fdr_exact(1, 8) - math.log2(n)

def epsilon(n, N):
    if n==1:
        return 0
    p = (n-1)/N
    return -(p/(1-p))*math.log2(p) - math.log2(1-p)

def es_uniform_loss(bits):
    N = 1<<(bits-1)
    def loss(n):
        return epsilon(n, N)
    return loss

def binary_entropy(p):
    if p==0:
        return 0
    return -p*math.log2(p) - (1-p)*math.log2(1-p)

def es_real_uniform_loss(bits):
    s = 1
    N = 1<<(bits-1)
    def loss(n):
        nonlocal s
        while s<N:
            s = s * 2
        p = (s%n)/s
        s = s/n
        return binary_entropy(p)
    return loss

def card_shuffling_loss(lossfn):
    def loss(size):
        nonlocal lossfn
        total = 0
        for i in range(1,size+1):
            total += lossfn(i)
        return total
    return loss

def card_shuffling_efficiency(lossfn):
    def efficiency(n):
        nonlocal lossfn
        output = 0
        loss = 0
        for i in range(1,n+1):
            output += math.log2(i)
            loss += lossfn(i)
        return output / (output + loss)
    return efficiency



knuth_yao_uniform_loss(5), von_neumann_uniform_loss(5)
es_uniform_loss(32)(5), es_real_uniform_loss(32)(5)
card_shuffling_loss(knuth_yao_uniform_loss)(52)

card_shuffling_efficiency(es_uniform_loss(32))(52)


0.9999999247888398

```
def expected_interval(p):
    def simulate(depth, range, delta):
        if range>=p or depth>10 or range+delta<=p:
            return depth * delta
        delta *= 0.5
        depth += 1
        return simulate(depth, range, delta) + simulate(depth, range+delta, delta)

    return simulate(0, 0, 1)

def expected_interval_loss(p):
    return expected_interval(p) - binary_entropy(p)
```

# Testing

In [330]:
U_s = 0
s = 1
N = 1<<31

U_s, s, U_n = generate_uniform(U_s, s, 6, N)

U_n

2

In [331]:
U_s, s, B = generate_bernoulli(U_s, s, 1, 3, N)
B

0

In [332]:
counts = [0,0]
for i in range(1,100):
    U_s, s, B = generate_bernoulli(U_s, s, 1, 6, N)
    counts[B] += 1

print(counts)

[86, 13]


In [333]:
make_distribution([1,2,3])

([0, 1, 1, 2, 2, 2], [0, 1, 3])

In [334]:
weights = [1,2,3]
outputs, offsets = make_distribution(weights)
U_s, s, x = generate_distribution(U_s, s, N, weights, outputs, offsets)

x

2

In [335]:
counts = [0,0,0]
for i in range(1,100):
    U_s, s, i = generate_distribution(U_s, s, N, weights, outputs, offsets)
    counts[i] += 1

print(counts)

[28, 28, 43]
